<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Data Connections Exercise

## Ask a Legal Research Assistant Bot about the US Constitution

Your function should do the following:

* Read the US_Constitution.txt file inside the some_data folder
* Split this into chunks (you choose the size)
* Write this to a ChromaDB Vector Store
* Use Context Compression to return the relevant portion of the document to the question

In [1]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 

In [10]:
def us_constitution_helper(question):
    '''
    Takes in a question about the US Constitution and returns the most relevant
    part of the constitution. Notice it may not directly answer the actual question!
    
    Follow the steps below to fill out this function:
    '''
    # Read apikey information for use when calling OpenAI libraries
    f = open('../openai_key.txt')
    api_key = f.read()
    
    # PART ONE:
    # LOAD "some_data/US_Constitution in a Document object
    loader = TextLoader('./some_data/US_Constitution.txt')

    data = loader.load()

    # PART TWO
    # Split the document into chunks (you choose how and what size)
    text_splitter = CharacterTextSplitter(separator="\n\n",chunk_size=1000)
    docs = text_splitter.split_documents(data)


    # PART THREE
    # EMBED THE Documents (now in chunks) to a persisted ChromaDB

    # Embedding function instantiated
    embedding_function = OpenAIEmbeddings(api_key=api_key)

    # Embed created document using above emedding function
    db_connection = Chroma.from_documents(docs, embedding_function,persist_directory='./us_constitution_speech_embedding_db')
    # Helpful to force a save
    db_connection.persist()

    # PART FOUR
    # Use ChatOpenAI and ContextualCompressionRetriever to return the most
    # relevant part of the documents.

    # Instantiate ChatOpenAI
    llm = ChatOpenAI(api_key=api_key)
    #  LLMChainExtractor uses an LLMChain to extract from each document only the statements that are relevant to the query. initialised by LLM
    compressor = LLMChainExtractor.from_llm(llm=llm)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db_connection.as_retriever())
    
    # Get the compressed docs based on questions. May return multiple instances
    compressed_docs = compression_retriever.get_relevant_documents(question)

    return compressed_docs[0].page_content

## Example Usage:

Notice how it doesn't return an entire Document of a large chunk size, but instead the "compressed" version!

In [11]:
print(us_constitution_helper("What is the 13th Amendment?"))

Created a chunk of size 1035, which is longer than the specified 1000
Created a chunk of size 1144, which is longer than the specified 1000
Created a chunk of size 1576, which is longer than the specified 1000
Created a chunk of size 2353, which is longer than the specified 1000
Created a chunk of size 1670, which is longer than the specified 1000
c:\Users\ZhiQiangQUEK\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\ZhiQiangQUEK\anaconda3\envs\llm\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported 

13th Amendment
Section 1
Neither slavery nor involuntary servitude, except as a punishment for crime whereof the party shall have been duly convicted, shall exist within the United States, or any place subject to their jurisdiction.
